In [2]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from itertools import count
import datetime
import re

def get_request_url(url):
    
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    req = urllib.request.Request(url,headers=headers)

    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            rcv = response.read()    
            
            return rcv
            
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def get_content(url):
    bs=BeautifulSoup(get_request_url(url),'html.parser')
    body=bs.find('div','_article_body_contents')
    
    gija=re.compile(r'\w+@\w+.')
    
    '''이미지 주석을 찾아서 리스트에 저장'''
    joosuck=[]
    try:
        for a in body.findAll('span','end_photo_org'):
            joosuck.append(a.text)
    except:
        joosuck=[]
        
    '''(기자명) + 기자 메일 앞까지 자르기 위해 역순으로 탐색'''
    index=10000
    for string in list(body.strings)[::-1]:
        if re.search(gija,string)!=None:
            index=list(body.strings).index(string)
            break
    
    '''이미지의 주석이 있는 경우 제거하고 body_content에 추가'''
    body_content=""
    flag=0
    for string in list(body.strings)[:index]:
        string=string.strip()
        if string!='':   # 공백 문자열 제거
            if(len(joosuck)!=0):
                for i in range(len(joosuck)):
                    if re.search(string, joosuck[i])==None:
                        flag=0
                    else:
                        flag=1
                        break
                if flag==0:
                    body_content += string+" "
            else:
                body_content += string + " "
    return body_content

def main():
    offi_name=input("신문사: ")
    date_start=input("시작 날짜: ")
    date_end=input("마지막 날짜: ")
    news_area=input("분야: ")
    sid_list={'정치':'100','경제':'101','사회':'102','생활/문화':'103','세계':'104','IT/과학':'105'}
    sid=sid_list[news_area]

    dt_index = pd.date_range(start=date_start, end=date_end)
    dt_list = dt_index.strftime("%Y%m%d").tolist()

    title=[]
    url=[]
    content=[]
    news_date=[]
    offi_name_list=[]
    sid_name=[]

    for date in dt_list:
        for page in count(1):
            url_page='https://news.naver.com/main/list.nhn?mode=LSD&mid=shm&sid1=%s&date=%s&page=%s' %(sid,date,str(page))
            bs=BeautifulSoup(get_request_url(url_page),'html.parser')
            headline_lis=bs.find('ul','type06_headline').find_all('li')
            for headline in headline_lis:
                headline_dd=headline.dl.find('dd')
                if headline_dd.find('span','writing').get_text()==offi_name:
                    headline_dt=headline.dl.find_all('dt')[-1]
                    try:
                        body_content=get_content(headline_dt.a['href'])
                        url.append(headline_dt.a['href'])
                        title.append(headline_dt.get_text().strip())
                        news_date.append(date)
                        offi_name_list.append(offi_name)
                        sid_name.append(news_area)
                        content.append(body_content)
                    except:
                        continue
                
            try:
                headline_lis2=bs.find('ul','type06').find_all('li')
                for headline in headline_lis2:
                    headline_dd=headline.dl.find('dd')
                    if headline_dd.find('span','writing').get_text()==offi_name:
                        try:
                            body_content=get_content(headline_dt.a['href'])
                            url.append(headline_dt.a['href'])
                            title.append(headline_dt.get_text().strip())
                            news_date.append(date)
                            offi_name_list.append(offi_name)
                            sid_name.append(news_area)
                            content.append(body_content)
                        except:
                            continue
                    
            except:
                print("page %s doesn't have 'type06' class" %page)
        
            if ((bs.find('div','paging').find('a',text='다음')==None) and (bs.find('div','paging').find('a',text=str(page+1))==None)):
                print("date=%s, page:%s complete[last page]" %(date,page))
                break
            else :
                print("date=%s, page:%s complete" %(date,page))
            
    data = {'신문사':offi_name_list, '날짜':news_date, '분야':sid_name, '기사제목':title, '본문':content, 'link':url}
    df=pd.DataFrame(data)
    df.to_csv('naver_news_crawling.csv', encoding="utf-8-sig", mode='w', index=True)  # 한자와 일부 특수문자 깨짐현상으로 cp949대신 utf-8-sig사용
    print("-----naver news crawling completion-----")

main()

신문사: 동아일보
시작 날짜: 20201001
마지막 날짜: 20201031
분야: 정치
date=20201001, page:1 complete
date=20201001, page:2 complete
date=20201001, page:3 complete
date=20201001, page:4 complete
date=20201001, page:5 complete
date=20201001, page:6 complete
date=20201001, page:7 complete
date=20201001, page:8 complete
date=20201001, page:9 complete
date=20201001, page:10 complete
date=20201001, page:11 complete
date=20201001, page:12 complete
date=20201001, page:13 complete
date=20201001, page:14 complete
date=20201001, page:15 complete
date=20201001, page:16 complete
date=20201001, page:17 complete
date=20201001, page:18 complete
date=20201001, page:19 complete
date=20201001, page:20 complete
date=20201001, page:21 complete
date=20201001, page:22 complete
date=20201001, page:23 complete
date=20201001, page:24 complete
date=20201001, page:25 complete
date=20201001, page:26 complete
date=20201001, page:27 complete
date=20201001, page:28 complete
date=20201001, page:29 complete
date=20201001, page:30 complete

date=20201005, page:40 complete
date=20201005, page:41 complete
date=20201005, page:42 complete
date=20201005, page:43 complete
date=20201005, page:44 complete
date=20201005, page:45 complete
date=20201005, page:46 complete
date=20201005, page:47 complete
date=20201005, page:48 complete
date=20201005, page:49 complete
date=20201005, page:50 complete
date=20201005, page:51 complete
date=20201005, page:52 complete
date=20201005, page:53 complete
date=20201005, page:54 complete
date=20201005, page:55 complete
date=20201005, page:56 complete
date=20201005, page:57 complete
date=20201005, page:58 complete
date=20201005, page:59 complete
date=20201005, page:60 complete
date=20201005, page:61 complete
date=20201005, page:62 complete
date=20201005, page:63 complete
date=20201005, page:64 complete
date=20201005, page:65 complete
date=20201005, page:66 complete
date=20201005, page:67 complete
date=20201005, page:68 complete
date=20201005, page:69 complete
date=20201005, page:70 complete
date=202

date=20201006, page:122 complete
date=20201006, page:123 complete
date=20201006, page:124 complete
date=20201006, page:125 complete
date=20201006, page:126 complete
date=20201006, page:127 complete
date=20201006, page:128 complete
date=20201006, page:129 complete
date=20201006, page:130 complete
date=20201006, page:131 complete
date=20201006, page:132 complete
date=20201006, page:133 complete
date=20201006, page:134 complete
date=20201006, page:135 complete
date=20201006, page:136 complete
date=20201006, page:137 complete
date=20201006, page:138 complete
date=20201006, page:139 complete
date=20201006, page:140 complete
date=20201006, page:141 complete
date=20201006, page:142 complete
date=20201006, page:143 complete
date=20201006, page:144 complete
date=20201006, page:145 complete
date=20201006, page:146 complete
date=20201006, page:147 complete
date=20201006, page:148 complete
date=20201006, page:149 complete
date=20201006, page:150 complete
date=20201006, page:151 complete
date=20201

date=20201007, page:170 complete
date=20201007, page:171 complete
date=20201007, page:172 complete
date=20201007, page:173 complete
date=20201007, page:174 complete
date=20201007, page:175 complete
date=20201007, page:176 complete
date=20201007, page:177 complete
date=20201007, page:178 complete
date=20201007, page:179 complete
date=20201007, page:180 complete
date=20201007, page:181 complete
date=20201007, page:182 complete
date=20201007, page:183 complete
date=20201007, page:184 complete
date=20201007, page:185 complete
date=20201007, page:186 complete
date=20201007, page:187 complete
date=20201007, page:188 complete
date=20201007, page:189 complete
date=20201007, page:190 complete
date=20201007, page:191 complete
date=20201007, page:192 complete
date=20201007, page:193 complete
date=20201007, page:194 complete
date=20201007, page:195 complete
date=20201007, page:196 complete
date=20201007, page:197 complete
date=20201007, page:198 complete
date=20201007, page:199 complete
date=20201

date=20201008, page:130 complete
date=20201008, page:131 complete
date=20201008, page:132 complete
date=20201008, page:133 complete
date=20201008, page:134 complete
date=20201008, page:135 complete
date=20201008, page:136 complete
date=20201008, page:137 complete
date=20201008, page:138 complete
date=20201008, page:139 complete
date=20201008, page:140 complete
date=20201008, page:141 complete
date=20201008, page:142 complete
date=20201008, page:143 complete
date=20201008, page:144 complete
date=20201008, page:145 complete
date=20201008, page:146 complete
date=20201008, page:147 complete
date=20201008, page:148 complete
date=20201008, page:149 complete
date=20201008, page:150 complete
date=20201008, page:151 complete
date=20201008, page:152 complete
date=20201008, page:153 complete
date=20201008, page:154 complete
date=20201008, page:155 complete
date=20201008, page:156 complete
date=20201008, page:157 complete
date=20201008, page:158 complete
date=20201008, page:159 complete
date=20201

...

date=20201023, page:98 complete
date=20201023, page:99 complete
date=20201023, page:100 complete
date=20201023, page:101 complete
date=20201023, page:102 complete
date=20201023, page:103 complete
date=20201023, page:104 complete
date=20201023, page:105 complete
date=20201023, page:106 complete
date=20201023, page:107 complete
date=20201023, page:108 complete
date=20201023, page:109 complete
date=20201023, page:110 complete
date=20201023, page:111 complete
date=20201023, page:112 complete
date=20201023, page:113 complete
date=20201023, page:114 complete
date=20201023, page:115 complete
date=20201023, page:116 complete
date=20201023, page:117 complete
date=20201023, page:118 complete
date=20201023, page:119 complete
date=20201023, page:120 complete
date=20201023, page:121 complete
date=20201023, page:122 complete
date=20201023, page:123 complete
date=20201023, page:124 complete
date=20201023, page:125 complete
date=20201023, page:126 complete
date=20201023, page:127 complete
date=2020102

date=20201026, page:55 complete
date=20201026, page:56 complete
date=20201026, page:57 complete
date=20201026, page:58 complete
date=20201026, page:59 complete
date=20201026, page:60 complete
date=20201026, page:61 complete
date=20201026, page:62 complete
date=20201026, page:63 complete
date=20201026, page:64 complete
date=20201026, page:65 complete
date=20201026, page:66 complete
date=20201026, page:67 complete
date=20201026, page:68 complete
date=20201026, page:69 complete
date=20201026, page:70 complete
date=20201026, page:71 complete
date=20201026, page:72 complete
date=20201026, page:73 complete
date=20201026, page:74 complete
date=20201026, page:75 complete
date=20201026, page:76 complete
date=20201026, page:77 complete
date=20201026, page:78 complete
date=20201026, page:79 complete
date=20201026, page:80 complete
date=20201026, page:81 complete
date=20201026, page:82 complete
date=20201026, page:83 complete
date=20201026, page:84 complete
date=20201026, page:85 complete
date=202

date=20201027, page:82 complete
date=20201027, page:83 complete
date=20201027, page:84 complete
date=20201027, page:85 complete
date=20201027, page:86 complete
date=20201027, page:87 complete
date=20201027, page:88 complete
date=20201027, page:89 complete
date=20201027, page:90 complete
date=20201027, page:91 complete
date=20201027, page:92 complete
date=20201027, page:93 complete
date=20201027, page:94 complete
date=20201027, page:95 complete
date=20201027, page:96 complete
date=20201027, page:97 complete
date=20201027, page:98 complete
date=20201027, page:99 complete
date=20201027, page:100 complete
date=20201027, page:101 complete
date=20201027, page:102 complete
date=20201027, page:103 complete
date=20201027, page:104 complete
date=20201027, page:105 complete
date=20201027, page:106 complete
date=20201027, page:107 complete
date=20201027, page:108 complete
date=20201027, page:109 complete
date=20201027, page:110 complete
date=20201027, page:111 complete
date=20201027, page:112 comp

date=20201028, page:155 complete
date=20201028, page:156 complete
date=20201028, page:157 complete
date=20201028, page:158 complete
date=20201028, page:159 complete
date=20201028, page:160 complete
date=20201028, page:161 complete
date=20201028, page:162 complete
date=20201028, page:163 complete
date=20201028, page:164 complete
date=20201028, page:165 complete
date=20201028, page:166 complete
date=20201028, page:167 complete
date=20201028, page:168 complete
date=20201028, page:169 complete
date=20201028, page:170 complete
date=20201028, page:171 complete
date=20201028, page:172 complete
date=20201028, page:173 complete
date=20201028, page:174 complete
date=20201028, page:175 complete
date=20201028, page:176 complete
date=20201028, page:177 complete
date=20201028, page:178 complete
date=20201028, page:179 complete
date=20201028, page:180 complete
date=20201028, page:181 complete
date=20201028, page:182 complete
date=20201028, page:183 complete
date=20201028, page:184 complete
date=20201

date=20201029, page:154 complete
date=20201029, page:155 complete
date=20201029, page:156 complete
date=20201029, page:157 complete
date=20201029, page:158 complete
date=20201029, page:159 complete
date=20201029, page:160 complete
date=20201029, page:161 complete
date=20201029, page:162 complete
date=20201029, page:163 complete
date=20201029, page:164 complete
date=20201029, page:165 complete
date=20201029, page:166 complete
date=20201029, page:167 complete
date=20201029, page:168 complete
date=20201029, page:169 complete
date=20201029, page:170 complete
date=20201029, page:171 complete
date=20201029, page:172 complete
date=20201029, page:173 complete
date=20201029, page:174 complete
date=20201029, page:175 complete
date=20201029, page:176 complete
date=20201029, page:177 complete
date=20201029, page:178 complete
date=20201029, page:179 complete
date=20201029, page:180 complete
date=20201029, page:181 complete
date=20201029, page:182 complete
date=20201029, page:183 complete
date=20201